# Task 1: PageRank Stability on Evolving Graphs
## Graph evolution and PageRank values comparison
### Joris & Abdel

### Imports and general set-up

In [1]:
import networkx as nx
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

Load edge list and create a graph

In [2]:
fh = open("canvas/hamster.edgelist", 'rb')
G = nx.read_edgelist(fh, create_using=nx.DiGraph())
fh.close()

In [3]:
np.random.seed(98)
random.seed(99)

In [4]:
def calc_pagerank(G_in, alpha = 0.85):
    return nx.pagerank(G_in, alpha=0.85)

In [5]:
pr_origin = calc_pagerank(G)

In [6]:
def create_dataframe(pr, G_in):
    df_edge_in = pd.DataFrame(list(G_in.in_degree()), columns=['node', 'in edges'])
    df_edge_out = pd.DataFrame(list(G_in.out_degree()), columns=['node', 'out edges'])
    df_rank = pd.DataFrame(list(pr.items()), columns=['node', 'score']).sort_values(by=['score'], ascending=False)
    df_temp = pd.merge(df_rank, df_edge_in, on='node')
    df_total = pd.merge(df_temp, df_edge_out, on='node')
    df_total.index = df_total.index + 1
    df_total.columns.name = 'rank'
    return df_total

In [179]:
df_origin = create_dataframe(pr_origin, G)
df_origin.head()

rank,node,score,in edges,out edges
1,404,0.042793,10,0
2,195,0.019961,80,1
3,77,0.018628,121,2
4,728,0.015530,10,0
5,36,0.011117,168,5


It looks like node 404 is the best ranked page, following by 195 and 77. This means that these pages should be shown at the top by search engines.

### <font color="red">To do: Update the PageRank value calculation and analysis in this notebook with the extended/corrected/completed <i>(this still needs to be done)</i> version of the notebook of 1a</font>

## 1b. Graph Evolution and Pagerank values comparison

In this section the effects of graph evolutions are going to be studied in relation to an evaluation of the stability of PageRank. In particular, various methodologies are going to be devised and exploited in which graphical represesentations of a social network are going to be altered by the removal and/or addition of nodes and edges in these graphs. The original graph $G$, represents a social network of friendships and familylinks between users of the website <a>hamsterster.com</a>. Various functions which make it possible to change this graph are going to be given and explained. Some of these functions focus on the addition or removal of edges, while other focus on nodes. Some of these functions are going to do make choices at random, while others are also going to exploit randomness, but proporotional to the node degree and other statistics. The choice is made to analyze the effects of the functions which evolve the graphs on the original graph $G$. So the evaluation of the various functions which add/remove graphs is going to be done starting from the full and original graph $G$ for each of the given functions. 

<i>Note: A social network would naturally be described with an undirected graph. The social network data is, however, treated as a combination of target and source id's which faciliate the usage of this data as a directed graph for the sake of implementing and testing graph evolutions methods to evaluate the stability of PageRank. No implications or conclusions should be directly related to the actual structure of the social networks of the website</i>

### A. Removing and adding edges uniformly at random

For $n$ number of nodes do the following:
* select 1 node uniformly at random
* add or remove an incoming/outgoing at random 

In [8]:
#add/remove edges for all the nodes uniformly at random
def random_edges_uniform_random(G_in, number_of_nodes = 1):
    list_of_nodes = list(G_in) # all the nodes
    # select uniformly at random nodes of which we are going to add/remove edges
    selected_nodes = list(np.random.choice(list_of_nodes, size = number_of_nodes, replace = False)) # default probability p is an uniform distribution
    
    for node in selected_nodes: 
        successors = list(G_in.successors(str(node))) # find the successors of this nodes
        predecessors = list(G_in.predecessors(str(node))) # find the predecessors of this node
        #find candidates for new edges
        unconnected_to = [n for n in list(G_in.nodes()) if not n in successors] # no outgoing edge to these nodes
        unconnected_from = [n for n in list(G_in.nodes()) if not n in predecessors] # no incoming edge from these nodes
        
        add = bool(random.getrandbits(1)) # randomly add or remove an edge of this node
        incoming =  bool(random.getrandbits(1)) # randomly add an outgoing/incoming edge
        if(add): # add an incoming/outgoing edge to node
            if(incoming): # add incoming edge
                if len(unconnected_from): #only add when unconnected_from is not empty
                    new = random.choice(unconnected_from)
                    G_in.add_edge(new, node)
                    print("\tnew edge:\t {} --> {}".format(new, node))
                    unconnected_from.remove(new)
                    predecessors.append(new)
            else: # add outgoing edge:
                if len(unconnected_to): #only add when unconnected_to is not empty
                    new = random.choice(unconnected_to)
                    G_in.add_edge(node, new)
                    print("\tnew edge:\t {} --> {}".format(node, new))
                    unconnected_to.remove(new)    
                    successors.append(new)
        else: # remove
            if(incoming): # remove incoming edge
                if len(predecessors): #only remove when predecessors is not empty
                    new = random.choice(predecessors)
                    G_in.remove_edge(new, node)
                    print("\tremove edge:\t {} --> {}".format(new, node))
                    predecessors.remove(new)
                    unconnected_from.append(new)
            else: # remove outgoing edge:
                if len(successors): #only remove when successors is not empty
                    new = random.choice(successors)
                    G_in.remove_edge(node, new)
                    print("\tremove edge:\t {} --> {}".format(node, new))
                    successors.remove(new)    
                    unconnected_to.append(new)
    return G_in

In [9]:
G_random_edges_uniform_random = random_edges_uniform_random(G.copy(), 100)

	remove edge:	 1612 --> 817
	remove edge:	 301 --> 303
	new edge:	 709 --> 1036
	new edge:	 1653 --> 2179
	new edge:	 2207 --> 1376
	remove edge:	 2412 --> 2413
	remove edge:	 648 --> 845
	new edge:	 889 --> 909
	new edge:	 1532 --> 2107
	new edge:	 541 --> 877
	remove edge:	 2384 --> 2354
	remove edge:	 2248 --> 305
	remove edge:	 522 --> 523
	remove edge:	 1887 --> 1889
	remove edge:	 697 --> 728
	remove edge:	 1801 --> 308
	new edge:	 2192 --> 1863
	remove edge:	 1631 --> 1638
	new edge:	 2328 --> 1903
	new edge:	 1292 --> 1467
	remove edge:	 1300 --> 249
	new edge:	 1810 --> 717
	new edge:	 728 --> 1889
	new edge:	 721 --> 1112
	remove edge:	 1123 --> 421
	remove edge:	 37 --> 60
	new edge:	 1234 --> 333
	new edge:	 189 --> 359
	remove edge:	 1724 --> 1725
	remove edge:	 964 --> 967
	new edge:	 1934 --> 825
	new edge:	 227 --> 460
	new edge:	 2049 --> 1817
	remove edge:	 470 --> 958
	new edge:	 522 --> 736
	remove edge:	 2094 --> 303
	remove edge:	 54 --> 121
	remove edge:	 830 -->

In [10]:
pr_random_edges_uniform_random = calc_pagerank(G_random_edges_uniform_random)
df_random_edges_uniform_random = create_dataframe(pr_random_edges_uniform_random, G_random_edges_uniform_random)
df_random_edges_uniform_random.head()

rank,node,score,in edges,out edges
1,404,0.042988,10,0
2,195,0.020129,80,1
3,77,0.018712,122,2
4,36,0.011565,168,5
5,192,0.009662,57,3


### B. Adding nodes uniformly at random (copying model)

For $n$ iterations do the following:
* Make a new node instance $n$
* with a uniform random distribution pick $k$ nodes in the original graph
* copy the incoming/outgoing edges of the $k$ nodes for $n$
* choose with an unifrom distribution another node $l$ and add its edges also to $n$

<i>The last step might seem redundant at the moment, but later when the the $k$ nodes are going to be chosen with at random but proportional to a certain statistic, it makes sense to have a step in which you pick another node $l$ that is chosen with the opposite property so that the generation/stability of communities is ensured (i.e. power-law degree). In this implementation. This step is omitted, but it will be thus added in the functions that take statistical measures into consideration when choosing nodes at random</i>

In [11]:
#randomly add and remove nodes
#Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_add_nodes_uniform(G_in, number_of_nodes = 1, k = 5):
    print("number of edges before :"+ str(len(G_in.edges())))
    for _ in range(number_of_nodes):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        #print("k = " + str(k))
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        # print("new node = " + str(new_node))
        
        list_of_nodes = list(G_in)  #create list of nodes
        
        G_in.add_node(str(new_node))   
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, replace = False) # k nodes with a uniform distribution
        
        for node in k_random_selected_nodes:
            #print("node in k_random_selected_nodes = " + str(node))
            successors = list(G_in.successors(str(node)))
            #print("succesors are " + str(successors))
            for node_to in successors:
                G_in.add_edge(new_node, node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(str(node)))
            for node_from in predecessors:
                G_in.add_edge(node_from, new_node) # add incoming edges
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [12]:
G_random_add_nodes_uniform = random_add_nodes_uniform(G.copy(), 100)

number of edges before :16631
number of edges after :17882


In [13]:
pr_random_add_nodes_uniform = calc_pagerank(G_random_add_nodes_uniform)
df_random_add_nodes_uniform = create_dataframe(pr_random_add_nodes_uniform, G_random_add_nodes_uniform)
df_random_add_nodes_uniform.head()

rank,node,score,in edges,out edges
1,404,0.040974,10,0
2,195,0.019240,82,1
3,77,0.018008,124,2
4,728,0.015021,10,0
5,135,0.010939,53,8


### C. Removal of nodes uniformly at random

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ uniformly at random (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [14]:
def random_removal_nodes_uniform(G_in, number_given = False, number_of_nodes = None):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    print("number of nodes before :"+ str(len(list(G_in))))
    list_of_nodes = list(G_in)
    selected_nodes = np.random.choice(list_of_nodes, size = n, replace = False)
    for m_remove in selected_nodes:
        G_in.remove_node(m_remove)
    print("number of nodes after :"+ str(len(list(G_in))))
    return G_in

In [15]:
G_random_removal_nodes_uniform = random_removal_nodes_uniform(G.copy(), True, 100)

number of nodes before :2426
number of nodes after :2326


In [16]:
pr_random_removal_nodes_uniform = calc_pagerank(G_random_removal_nodes_uniform)
df_random_removal_nodes_uniform = create_dataframe(pr_random_removal_nodes_uniform, G_random_removal_nodes_uniform)
df_random_removal_nodes_uniform.head()

rank,node,score,in edges,out edges
1,404,0.042501,9,0
2,195,0.020760,78,1
3,77,0.019746,117,2
4,728,0.017544,10,0
5,36,0.011739,167,5


## Graph evolution methodologies using statistical measures

### D. Removal of nodes at random but proportional to the degree of the nodes

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ at random, but proportional to the in degree's of the nodes (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [17]:
def random_node_removals_proportional_degree(G_in, number_given = False, number_of_nodes = None, in_degree = True):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    print("number of nodes before :"+ str(len(list(G_in))))
    list_of_nodes = list(G_in)
    if (in_degree):
        degrees = dict(G_in.in_degree()).values() # in_degrees of all the nodes
    else: # out_degree
        degrees = dict(G_in.out_degree()).values() # in_degrees of all the nodes
    prob_degree = [float(i)/sum(degrees) for i in degrees] # probabilities proportional to degree
    
    selected_nodes = np.random.choice(list_of_nodes, size = n, replace = False, p = prob_degree)
    for m_remove in selected_nodes:
        G_in.remove_node(m_remove)
    print("number of nodes after :"+ str(len(list(G_in))))
    return G_in

In [18]:
G_random_node_removals_proportional_indegree = random_node_removals_proportional_degree(G.copy(), True, 100, True)
pr_random_node_removals_proportional_indegree = calc_pagerank(G_random_node_removals_proportional_indegree)
df_random_node_removals_proportional_indegree = create_dataframe(pr_random_node_removals_proportional_indegree,
                                                                 G_random_node_removals_proportional_indegree)
df_random_node_removals_proportional_indegree.head()

number of nodes before :2426
number of nodes after :2326


rank,node,score,in edges,out edges
1,404,0.032206,8,0
2,728,0.020527,10,0
3,34,0.013388,122,4
4,181,0.012316,87,10
5,126,0.012174,45,1


Let's have a look at the hundred nodes that are actually removed by the algorithm. The assumption is that the average in-degree of these 100 nodes is higher than the average node degree of the original graph $G$ since the nodes are removed at random, but proportional to their in-degree.

In [38]:
avg_node_degree_full_graph = df_origin["in edges"].mean()
avg_node_degree_graph_removed_nodes = df_random_node_removals_proportional_indegree["in edges"].mean()
round(avg_node_degree_full_graph, 6), round(avg_node_degree_graph_removed_nodes, 6)

(6.855317, 5.036543)

Since the average node in-degree dropped significantly, we can indeed conclude that the function removed nodes at random, but proportional to their node degrees since th drop in the average node degree of all the nodes.

In [20]:
nodes_original_graph = set(df_origin.node.values)
nodes_evolved_graph = set(df_random_node_removals_proportional_indegree.node.values)
removed_nodes = pd.DataFrame(list(nodes_original_graph.difference(nodes_evolved_graph)))
removed_nodes = pd.merge(df_origin, removed_nodes, left_on = 'node', right_on = 0)
avg_node_degree_removed_nodes = removed_nodes["in edges"].mean()
avg_node_degree_removed_nodes

39.1

And as already confirmed in the previous cell we can indeed conclude that the nodes that are removed have a significant higher node in-degree (on-average) than the average of the nodes in the original graph. We can thus indeed conclude that the function removed nodes at random, but proportional to their node degrees since th drop in the average node degree of all the nodes.

Let's use our knowledge of the inner-workings of PageRank to state hyptothesis about the changes of PageRank of the (nodes in the) evolved graph, compared to the original graph. We know that the PageRank score of a node is influenced by the number (the set) of incoming edges, the number of outbound links of the source nodes of these edges and the score of the particular source nodes. In addition, the damping factor is used in the calculations, but this is kept the same in this PageRank stability analysis so will be left out of consideration.

Let's have a look at what has happened to the PageRank scores after the removal of nodes at random, but proportional to the in-degree. As we have seen earlier, the nodes that were removed had on average, a much higher in-degree. Considering the PageRank formula, there is a high chance that nodes with a high number of incoming edges have a relatively high score. But this is affected by the quality of the nodes sourcing the incoming edge. In other words, even if a certain node $n_1$ has a high number of incoming edges than a node $n_2$, but the nodes connected to $n_2$ (have an outgoing edge to $n_2$ have relatively less outgoing links and a relatively higher score, it could be that the score of $n_2$ is higher than $n_1$.

Let's assess whether the nodes that were removed with a relaltively high in-degree compared to the rest of the nodes, have had a negative effect on the average score of the nodes in the graph. Considering the elabaroation that is given in the previous paragraph, this is affected by the number of outgoing links of the nodes that had an edge to the removed nodes and the score of these nodes.

In [37]:
avg_score_removed_nodes = removed_nodes.score.mean()
avg_score_full_graph = df_origin.score.mean()
round(avg_score_removed_nodes, 6), round(avg_score_full_graph, 6), round(avg_score_removed_nodes / avg_score_full_graph, 2)

(0.001643, 0.000412, 3.99)

We see that the the nodes that were removed indeed have a higher score (see factor) than the average score in the original graph of all the nodes.The nodes that had an outgoing edge to the nodes that were removed thus did not have such high number of total outgoing links that it affected the score of the removed nodes in such a way that the average score is not higher than that of the average node in the original graph.

Let's have a look at what happens with the PageRank scores when we remove nodes at random, but proportional to their out-degrees. When you look at the formula of PageRank, you see that an edge from a node with a lot of outoging links will make no significant contribution to the score of the particular node (when compared to a node of the same score with less outgoing links). The assumption is, therefore, that unlike in the previous case, the average PageRank score in the graph in which nodes are removed, will hardly differ from the average PageRank score in the original graph.

In [22]:
G_random_node_removals_proportional_outdegree = random_node_removals_proportional_degree(G.copy(), True, 100, False)
pr_random_node_removals_proportional_outdegree = calc_pagerank(G_random_node_removals_proportional_outdegree)
df_random_node_removals_proportional_outdegree = create_dataframe(pr_random_node_removals_proportional_outdegree,
                                                                 G_random_node_removals_proportional_outdegree)
df_random_node_removals_proportional_outdegree.head()

number of nodes before :2426
number of nodes after :2326


rank,node,score,in edges,out edges
1,404,0.043407,10,0
2,195,0.020449,73,1
3,77,0.018800,108,2
4,728,0.015592,10,0
5,36,0.011427,151,5


In [23]:
avg_score_graph_removed_nodes = df_random_node_removals_proportional_outdegree.score.mean()
avg_score_full_graph = df_origin.score.mean()
avg_score_graph_removed_nodes, avg_score_full_graph, round(avg_score_graph_removed_nodes / avg_score_full_graph, 2)

(0.0004299226139295095, 0.00041220115416324565, 1.04)

And indeed, there is no significant differences in the average PageRank scores of the full network when removing nodes at random, but proportional to the out degree of the nodes (i.e. nodes with a higher out-degree have a higher probability to be removed at random)

### E. Removal of nodes at random but proportional to the hubs/authorithy measures (HITS) of nodes

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ at random, but proportional to HITS measures (i.e. hub or authority) of the nodes (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [73]:
def random_node_removals_proportional_HITS(G_in, authorithy = False, number_given = False, number_of_nodes = None):
    print("number of nodes before: " + str(len(list(G_in))))
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    for i in range(0, n):
        list_of_nodes = list(G_in)
        #print(int((i / n) * 100), "%") # progress (nx.hits(Graph) takes some time depending on the graph size)
        if (authorithy):
            p = list(nx.hits(G_in)[1].values()) # probabilities proportional to authority of nodes
        else: # hub
            p = list(nx.hits(G_in)[0].values()) # probabilities proportional to hub of nodes
        node_remove = np.random.choice(list_of_nodes, p = p)
        G_in.remove_node(node_remove)
    print("number of nodes after: " + str(len(list(G_in))))
    return G_in

In [75]:
G_random_node_removals_proportional_authority = random_node_removals_proportional_HITS(G.copy(), True, True, 100)

number of nodes before: 2426
number of nodes after: 2326


In [76]:
pr_random_node_removals_proportional_authority = calc_pagerank(G_random_node_removals_proportional_authority)
df_random_node_removals_proportional_authority = create_dataframe(pr_random_node_removals_proportional_authority,
                                                             G_random_node_removals_proportional_authority)
df_random_node_removals_proportional_authority.head()

rank,node,score,in edges,out edges
1,404,0.029291,7,0
2,77,0.016805,94,1
3,728,0.016235,9,0
4,923,0.013499,6,0
5,899,0.012292,8,0


This will be a rather interesting analysis of the PageRank algorithm and its stability. Note that the previous function removes nodes at random, but proportional to algorithms acquired from the HITS algorithm. In particular, the funtion removes nodes at random proportional to their $hub$ or $authorithy$ values. Why is this interesting one may ask. A good hub reprresents a page (node) that points to many other pages, while a good authority represents a page that was linked by many different hubs. So there is definitely a relation between the HITS measures and the PageRank scores. While we could explain this relation in more detail, it is more fun to show this relation with evaluations of the node removals according to the HITS measurs, considering the avaialability of the data.

The node removal of 100 nodes that was just executed was at random, but proportional to the authorithy values of the nodes. This means that nodes with a higher authority value will have a higher chance of being removed. In other words, nodes that were referred to by many other hubs (i.e. incoming edges) will have had a higher chance of being removed. We have seen earlier on that the in general (dependending on the explained pecularities), the average PageRank scores of nodes (when $n$ is large enough) will be higher than the average PageRank score in the original graph, when the nodes are removed at random but proportional to this measure. Let's see of the function does what it should do and whether this is indeed the case.

In [77]:
avg_node_degree_graph_removed_nodes = df_random_node_removals_proportional_authority["in edges"].mean()
round(avg_node_degree_full_graph, 6), round(avg_node_degree_graph_removed_nodes, 6)

(6.855317, 5.088994)

First, note that it seems that that function indeed removed nodes at random but proportional to the authority values since the average in-degree value has dropped significantly after removal of the nodes. Which makes sense since the authority is an indication of how many pages (i.e. nodes in our context) linked to that particular node.

In [78]:
nodes_evolved_graph = set(df_random_node_removals_proportional_authority.node.values)
removed_nodes = pd.DataFrame(list(nodes_original_graph.difference(nodes_evolved_graph)))
removed_nodes = pd.merge(df_origin, removed_nodes, left_on = 'node', right_on = 0)
avg_node_degree_removed_nodes = removed_nodes["in edges"].mean()
avg_node_degree_removed_nodes

37.64

And note once again that the average in degree is much higher than the average in degree of the original graph. The function does what it should do. There is a strong relation with the previous analyis due to the strong relation between the HITS measures and the in/out-degrees of the nodes. We have shown that some statistics lead to the same conclusion. Let's devise an other method to evaluate the stability of the PageRank values after removal of the nodes according to their hub values. Remember, a good hub reprresents a page (node) that points to many other pages.

In [79]:
G_random_node_removals_proportional_hub = random_node_removals_proportional_HITS(G.copy(), False, True, 100)

number of nodes before: 2426
number of nodes after: 2326


In [100]:
pr_random_node_removals_proportional_hub = calc_pagerank(G_random_node_removals_proportional_hub)
df_random_node_removals_proportional_hub = create_dataframe(pr_random_node_removals_proportional_hub,
                                                             G_random_node_removals_proportional_hub)
df_random_node_removals_proportional_hub.head()

rank,node,score,in edges,out edges
1,404,0.041081,10,0
2,195,0.019682,72,1
3,77,0.017496,107,2
4,728,0.016139,10,0
5,36,0.011072,137,5


Like with the removal of nodes proportional to the out-degree (at random), the assumption is that removal of nodes proportional to their hub values, will have no significant effect on the PageRank values of the nodes in the the graph of the nodes that remain in the graph. We are, however, going to show this with a different method.

Let's evaluate the absolute change of the PageRank scores of the nodes that remain in the graph, once nodes are removed at random but proportional to their hub values. The hypothesis is already stated, let's see whether it holds up. We are going to form  a new DataFrame in which we have the new and old PageRank scores (i.e. before and after removal). And then going to add a new column with the %-change of the PageRank scores.

In [101]:
df_random_node_removals_proportional_hub.rename(columns = {'score': 'new_score', 'in edges': 'new_in_edges',
                                                           'out edges': 'new_out_edges'}, inplace=True)

In [102]:
df_random_node_removals_proportional_hub.head()

rank,node,new_score,new_in_edges,new_out_edges
1,404,0.041081,10,0
2,195,0.019682,72,1
3,77,0.017496,107,2
4,728,0.016139,10,0
5,36,0.011072,137,5


In [136]:
df_compare_scores = pd.merge(df_random_node_removals_proportional_hub, df_origin, on = 'node')
df_compare_scores["percentage_change"] = df_compare_scores.apply(lambda row: ((row.new_score - row.score) / row.score) * 100, axis = 1)
df_compare_scores["abs_percentage_change"] = df_compare_scores.apply(lambda row: abs(row.percentage_change), axis = 1)
print("top 10: ", df_compare_scores.head(10).abs_percentage_change.mean(),
      "\ntop 50: ", df_compare_scores.head(50).abs_percentage_change.mean(), 
      "\ntop 100: ", df_compare_scores.head(100).abs_percentage_change.mean(), 
      "\nwhole graph: ", df_compare_scores.abs_percentage_change.mean())

top 10:  2.9011196235737113 
top 50:  5.342112970287124 
top 100:  8.706663303998335 
whole graph:  8.077092153256247


First note that the average change of the PageRank scores in the graph is not significant (single digit point percentage change), but what is even more interesting and scientifically interesting and explainable, is the fact that the change of the scores is on average less for nodes that had already a relatively high score to begin with. This can be seen in the previous print statement, we see that the average change of the scores (in point percentage) seems to be correlated with the original score of the node. Note that in addition, the number of total pages is reduced by 100 ($N$ in the PageRank) formula, which also has an effect on the new scores.

To give a simplified explanation of the PageRank explanation (left out some assumptions such as no outbound edges = edge to all other nodes (pages) in the graph) let's have a look what is happening more or less under the hood. We have $PRs(n) = \dfrac{1 - d}{N} + d \cdot \sum_{m \, \in \, predecessors(n)} \dfrac{PRs(m)}{\mid\,successors(m)\,\mid}$, <br>
where $N =$ the number of pages (reduced by 100), $d$ is the damping factor (remained constant) and the notation $\mid\,successors(m)\,\mid$ is used to denote the <i>number</i> of successors of node $m$ (i.e. the number of outgoing edges of node $m$). So even if a node is not directly affected by removal of nodes (i.e. it keeps its edges as the nodes that were removed had no direct connection to it), its score will still change due to the fact that $N$ has changed and the fact that the score, or the number of outgoing edges of its predecessors could have changed. These are the things to take into consideration when looking at the changes and the statement that a single percentage change is not large for removal of 100 nodes.

And now back to the interesting correlation. It can be explained quite simply. Let us have a node $x$ with a relatively high PageRank score. If we would remove one its predecessors $y$ with a high number of outgoing edges (as is the case with the random removal of nodes proportional to the value of the hub measure), its effect on the score of node $x$ would not be significant or as high as the removal of a node $z$ with a relatively low amount of outgoing edges compared to $y$ but with the same score, its effect will be less significant as the number of outgoing edges is the denominator in the summation. And the contribtion of this node to the total sum of all the PageRank scores divided by the number of successors, of the predecessors would be small to begin with. This is a possible explanation (there can be other causes) of the correlation that has been shown. So in removal of nodes proportional to the hub values we have seen some interesting evaulations of the stability of the PageRank scores of the nodes in the graph. 

### F. Addition of nodes at random but proportional to the degree of the nodes

Let $n$ represent the number of nodes that should be added to the network. If the $number\_of\_nodes$ parameter is given then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Make a new node instance $n$
* with a probability distribution proportional to the in-degree of nodes pick $k$ nodes in the original graph
* copy the incoming/outgoing edges of the $k$ nodes for $n$
* Pick a node $l$ (at random) in the graph in the graph which had a relatively low probability in the second step and repeat step 3 for this node

In [165]:
type(np.random.choice(list(G), size = 5, replace = False)[0])

numpy.str_

In [215]:
# Inspired by the Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_node_additions_proportional_in_degree(G_in, number_given = False, in_degree = True, number_of_nodes = 1, k = 5):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    print("number of edges before :"+ str(len(G_in.edges())))
    for i in range(n):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        list_of_nodes = list(G_in)  #create list of nodes
        if (in_degree):
            degrees = dict(G_in.in_degree()).values() # in_degrees of all the nodes
        else: # out-degree
            degrees = dict(G_in.out_degree()).values() # out_degrees of all the nodes
        prob_degree = [float(i)/sum(degrees) for i in degrees] # probabilities proportional to degree
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, p = prob_degree, replace = False) # selecte k nodes proportional to chosen measure
        
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        G_in.add_node(str(new_node))
        
        for node in k_random_selected_nodes:
            successors = list(G_in.successors(str(node)))
            for node_to in successors:
                G_in.add_edge(str(new_node), node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(str(node)))
            for node_from in predecessors:
                G_in.add_edge(node_from, str(new_node)) # add incoming edges
        
        # pick one node that has a low probability (relatively low number of incoming edges)
        non_zero_probs = [i for i in prob_degree if i != 0.0]
        highest_chance_nodes = np.random.choice(list_of_nodes, p = prob_degree, 
                                                size = (len(non_zero_probs) - 1), replace = False)
        
        node_to_add = str(random.sample(set(list_of_nodes).difference(set(highest_chance_nodes)), 1)[0]) # low prob node
        successors = list(G_in.successors(node_to_add)) # successors of the node
        predecessors = list(G_in.predecessors(node_to_add)) # predecessors of the node 
        
        succ_current_node = list(G_in.successors(str(new_node))) # find the successors of the new node 
        pred_current_node = list(G_in.predecessors(str(new_node))) # find the predecessors of the new node
                                 
        # remove nodes to which the new node is already connected from the successors/predecessors list
        successors = [n for n in successors if not n in succ_current_node]
        predecessors = [n for n in predecessors if not n in pred_current_node]
                                 
        for node_to in successors:
            G_in.add_edge(str(new_node), node_to) # add outgoing edges
        for node_from in predecessors:
            G_in.add_edge(node_from, str(new_node)) # add incoming edges
            
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [214]:
G_random_node_additions_proportional_in_degree = random_node_additions_proportional_in_degree(G.copy(), True, True, 100, 5)

number of edges before :16631
number of edges after :23960


In [216]:
pr_random_node_additions_proportional_in_degree = calc_pagerank(G_random_node_additions_proportional_in_degree)
df_random_node_additions_proportional_in_degree = create_dataframe(pr_random_node_additions_proportional_in_degree,
                                                             G_random_node_additions_proportional_in_degree)
df_random_node_additions_proportional_in_degree.head(5)

rank,node,score,in edges,out edges
1,404,0.025541,17,0
2,728,0.012130,12,0
3,195,0.010145,103,1
4,2484,0.010145,103,3
5,2452,0.009196,161,15


This is going to be an interesting analysis where a lot of our knowledge about the workings of PageRank is going to be combined with correlations found in the analysis of previous functions. Let us first evaluate what is happening in the function. An $n$ number of nodes are added by first selecting $k$ nodes at random, but proportional to their in degrees and we copy all its edges. And then we do the opposite and select at random one node that has a relatively low probability (acquired from its low relatively in-degree) and copy its node (inspired by the Edge Copying Model, Kleinberg et al.).

We have seen in the analysis of function $D$ that in general, if we select nodes at random, but proportional to their in-degree, the average score and in-degree of this group is relatively higher than the the rest of the network when $n$ is large enough (guarrantees the insigniciance of cases where the score is affected by the number of outgoing links / score of the nodes).

So if we select $k$ nodes at random, but proportional to their in-degree, there is a high chance that they have a relatively high score. The PageRank score of the new node $n$ is calculated using, among other things, the score of nodes of its incoming edges divided by the number of outgoing links of those nodes. So there is a high chance that the $n$ new nodes have also a relatively high score compared to the average in the original graph (before the addition). Note that in this case, $N$ has dropped, which also has an effect on the PageRank scores since the first term becomes smaller. Let's see whether this hypothesis holds up in the experimental evaluation.

In [244]:
nodes_evolved_graph = set(df_random_node_additions_proportional_in_degree.node.values)
added_nodes = pd.DataFrame(list(nodes_evolved_graph.difference(nodes_original_graph)))
added_nodes = pd.merge(df_random_node_additions_proportional_in_degree, added_nodes, left_on = 'node', right_on = 0)
avg_node_score_added_nodes = added_nodes["score"].mean()
round(avg_node_score_added_nodes, 6)

0.001649

In [233]:
print(round(avg_score_full_graph, 6))
print("ratio: ", round(avg_node_score_added_nodes / avg_score_full_graph, 6))

0.000412
ratio:  4.00093


And we can indeed see that the scores of the average score is on average much higher than the average score in the original graph. Which is a correlation which seems to be in line with our hypothesis. Now another assumption is that the average PageRank has dropped, since we have added nodes that got incoming edges from nodes chosen at rando, but proportional to their in-degrees. This means that the nodes that were pointing to randomly selected nodes get more outgoing links, which affects the score of the selected nodes. Let's see if this is indeed the case.

In [237]:
check_new_score_original_nodes = pd.DataFrame(list(nodes_original_graph)) # omit new nodes, as this effects the results
check_new_score_original_nodes = pd.merge(df_random_node_additions_proportional_in_degree, check_new_score_original_nodes,
                                          left_on = 'node', right_on = 0)
avg_score_orginal_nodes = check_new_score_original_nodes.score.mean()
print("new average score original network: ", avg_score_orginal_nodes)
print("ratio drop: ", avg_score_orginal_nodes / avg_score_full_graph)

new average score original network:  0.0003442214381636475
ratio drop:  0.8350812089849804


While there are also othe factors which affect this drop (e.g. larger $N$ and backwards-propogation of changed scores), the PageRank scores of the nodes in the original network indeed seem to have taken a hit (no pun inteded/reference to next section).

### G. Addition of nodes at random but proportional to the hubs/authorithy measures (HITS) of nodes

Let $n$ represent the number of nodes that should be added to the network. If the $number\_of\_nodes$ parameter is given then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Make a new node instance $n$
* with a probability distribution proportional to one of the HITS measures (hub- or authority value) of nodes pick $k$ nodes in the original graph
* copy the incoming/outgoing edges of the $k$ nodes for $n$
* Pick a node $l$ (at random) in the graph in the graph which had a relatively low probability (due to a low value) in the second step and repeat step 3 for this node

In [238]:
#Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_node_additions_proportional_HITS(G_in, authority = False, number_given = False, number_of_nodes = 1, k = 5):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    print("number of edges before :"+ str(len(G_in.edges())))
    for _ in range(n):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        list_of_nodes = list(G_in)  #create list of nodes
        if (authority):
            p = list(nx.hits(G_in)[0].values())
        else: # hub
            p = list(nx.hits(G_in)[1].values())
        
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, p = p, replace = False) # selecte k nodes proportional to chosen measure
        
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        G_in.add_node(str(new_node))
        
        for node in k_random_selected_nodes:
            successors = list(G_in.successors(str(node)))
            for node_to in successors:
                G_in.add_edge(str(new_node), node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(str(node)))
            for node_from in predecessors:
                G_in.add_edge(node_from, str(new_node)) # add incoming edges
        
        # pick one node that has a low probability (relatively low number of incoming edges)
        non_zero_probs = [i for i in p if i != 0.0]
        highest_chance_nodes = np.random.choice(list_of_nodes, p = p, 
                                                size = (len(non_zero_probs) - 1), replace = False)
        
        node_to_add = random.sample(set(list_of_nodes).difference(set(highest_chance_nodes)), 1)[0] # low prob node
        successors = list(G_in.successors(node_to_add)) # successors of the node
        predecessors = list(G_in.predecessors(node_to_add)) # predecessors of the node 
        
        succ_current_node = list(G_in.successors(str(new_node))) # find the successors of the new node 
        pred_current_node = list(G_in.predecessors(str(new_node))) # find the predecessors of the new node
                                 
        # remove nodes to which the new node is already connected from the successors/predecessors list
        successors = [n for n in successors if not n in succ_current_node]
        predecessors = [n for n in predecessors if not n in pred_current_node]
                                 
        for node_to in successors:
            G_in.add_edge(str(new_node), node_to) # add outgoing edges
        for node_from in predecessors:
            G_in.add_edge(node_from, str(new_node)) # add incoming edges
            
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [248]:
G_random_node_additions_proportional_authority = random_node_additions_proportional_HITS(G.copy(), True, True, 100, 5)

number of edges before :16631
number of edges after :21239


In [249]:
pr_random_node_additions_proportional_authority = calc_pagerank(G_random_node_additions_proportional_authority)
df_random_node_additions_proportional_authority = create_dataframe(pr_random_node_additions_proportional_authority, G_random_node_additions_proportional_authority)
df_random_node_additions_proportional_authority.head()

rank,node,score,in edges,out edges
1,404,0.040836,10,0
2,195,0.020556,100,1
3,77,0.018731,150,2
4,728,0.015167,10,0
5,36,0.011156,189,6


We follow the same scheme as in the previous graph changin method.  An $n$ number of nodes are added by first selecting $k$ nodes at random, but proportional to their authority values (could also pick hub) and we copy all its edges. And then we do the opposite and select at random one node that has a relatively low probability (acquired from its low authority value) and copy its node (inspired by the Edge Copying Model, Kleinberg et al.).

Now if we copy the incoming edges of $n$ at random chosen nodes that were chosen proportionally to their authority values, we know that if we pick $n$ large enough, the average number of incoming edges of these $n$ nodes will be relatively high compared to the rest of the network, since a good authority (i.e. a relatively high authority value) represents a node that was linked by many different hubs. We, however, have to see whether the quality/scores of the nodes linking to the $m$ randomly chosen nodes are relatively high. If this is indeed the case we'll see a relatively high score of the added nodes.

In [250]:
nodes_evolved_graph = set(df_random_node_additions_proportional_authority.node.values)
added_nodes = pd.DataFrame(list(nodes_evolved_graph.difference(nodes_original_graph)))
added_nodes = pd.merge(df_random_node_additions_proportional_in_degree, added_nodes, left_on = 'node', right_on = 0)
avg_node_score_added_nodes = added_nodes["score"].mean()
round(avg_node_score_added_nodes, 6)

0.001649

In [251]:
print(round(avg_score_full_graph, 6))
print("ratio: ", round(avg_node_score_added_nodes / avg_score_full_graph, 6))

0.000412
ratio:  4.00093


And like with the nodes that were added that copied edges from nodes that were chosen at random but proportional to their in-degrees, we see that the usage of authority values lead to the same result.

We have seen various functions that changed the original graph $G$ by the addition and/or removal of nodes or edges. In some of these functions the eelction of the nodes or edges was done uniformly at random, while in other function it was done proportional to the node degree and other statistics. We have stated hypotheses about the nodes that were added/removed and have evaluated whether this actually happened in the experiments done with the functions using $G$. In addition, hypothesis were stated about the stability of the PageRank scores of the original nodes in $G$ when $G$ was changed according to stastical measures. We know have a rough idea of when PageRank scores will change siginificantly, and when not. This knowledge can be tested and exploited in the analysis of data of actual webpages on the Internet.